# Lab 3

The illusion of conversation, and multi-shot prompting

In [1]:
from dotenv import load_dotenv
from litellm import completion
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic.*")
load_dotenv(override=True)

True

In [2]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi there!"}
]

In [3]:
response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [4]:
messages = [
    {"role": "user", "content": "My name is George."}
]
response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

Hello, George! How can I assist you today?


In [5]:
messages = [
    {"role": "user", "content": "What's my name?"}
]
response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

I don't have access to your personal information unless you share it with me. What would you like me to call you?


In [7]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Hi there! How can I help you today?"},
    {"role": "user", "content": "My name is George."},
    {"role": "assistant", "content": "Hi George! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
]

response = completion(model="gpt-4.1-mini", messages=messages)
print(response.choices[0].message.content)

Your name is George. How can I assist you further, George?


## Explanation

The reason I show you this is to emphasize that every call to the LLM is completely stateless.

The sense that we are engaged in a chat with the LLM that takes place over several minutes is an illusion, enabled by the trick of passing in the entire conversation so far.

In [8]:
# Let's look at the cost - this is the cost of the entire conversation
# Any guesses on how much this is?

print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 65
Output tokens: 14
Total cost: 0.0048 cents


## Inference time "training" with multi-shot prompting

In [9]:
system_prompt = """
You are a helpful assistant that answers questions in English and then in Spanish.
"""
user_prompt = "What's the meaning of life?"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
response = completion(model="gpt-4.1-mini", messages=messages)
display(Markdown(response.choices[0].message.content))


The meaning of life is a deeply philosophical question that varies depending on individual beliefs, cultures, and perspectives. Some see it as seeking happiness, fulfilling personal goals, building relationships, or contributing to others. Others find meaning through spirituality, religion, or the pursuit of knowledge. Ultimately, the meaning of life is a personal journey and can be different for each person.

El significado de la vida es una pregunta profundamente filosófica que varía según las creencias individuales, las culturas y las perspectivas. Algunos lo ven como la búsqueda de la felicidad, el cumplimiento de metas personales, la construcción de relaciones o la contribución a los demás. Otros encuentran significado a través de la espiritualidad, la religión o la búsqueda del conocimiento. En última instancia, el significado de la vida es un viaje personal y puede ser diferente para cada persona.

In [10]:
system_prompt = """
You are a helpful assistant that answers questions in English and then in Spanish.

For example, if the user asks 'What is 2+2' then you would answer as follows:

## My reply in English is:
Four

## My reply in Spanish is:
Cuatro

"""
user_prompt = "What's the meaning of life?"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
response = completion(model="gpt-4.1-mini", messages=messages)
display(Markdown(response.choices[0].message.content))

## My reply in English is:
The meaning of life is a profound and philosophical question that has been explored by many cultures and thinkers throughout history. Generally, it can be understood as the pursuit of happiness, purpose, love, knowledge, and fulfillment. Different people and philosophies may have different interpretations, but often it revolves around finding meaning through relationships, personal growth, and contributing to something greater than oneself.

## My reply in Spanish is:
El significado de la vida es una pregunta profunda y filosófica que ha sido explorada por muchas culturas y pensadores a lo largo de la historia. Generalmente, se puede entender como la búsqueda de la felicidad, el propósito, el amor, el conocimiento y la realización personal. Diferentes personas y filosofías pueden tener interpretaciones distintas, pero a menudo gira en torno a encontrar sentido a través de las relaciones, el crecimiento personal y la contribución a algo más grande que uno mismo.

## Is this "training"?

It appears that the model _learned_ the format that we wanted, based on the example given.

But there was no training going on in the Data Science sense; the trillions of parameters in gpt-4.1-mini weren't changed.

It's simply that when predicting the next tokens to come after our question, the model is more likely to predict tokens that would follow the same pattern as the examples in the input.